In [25]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sci
from scipy.special import factorial 

# Factoriales

In [31]:
def normalize(n):
    expo=np.floor(np.log10(n))
    valor=(n/(10**expo))
    return valor,expo
normalize=np.vectorize(normalize)


def fac(N):
  ran = np.arange(1,2*N,1,dtype=np.float128)
  s_up = np.zeros((N,2),dtype=np.float128)
  s_dw = np.zeros((N,2),dtype=np.float128)
  
  n, expo=normalize(ran)  

  ex=np.zeros(np.size(n),dtype=np.float128)
  x=np.zeros(np.size(n),dtype=np.float128)

  for ii in range(np.size(n)):
    if ii == 0:
      x[ii]=1
    else:
      ex[ii]=expo[ii]+ex[ii-1]
      x[ii]=n[ii]*x[ii-1]
      if x[ii]>=10:
        ex[ii]=ex[ii]+(np.floor(np.log10(x[ii])))
        x[ii]=x[ii]/10**(np.floor(np.log10(x[ii])))

  s_dw[:, 0] =  x[:N:1]
  s_dw[:, 1] =  ex[:N:1]
  s_up[:, 0] =  np.concatenate((x[0], x[1:2*N-1:2]), axis=None)
  s_up[:, 1] =  np.concatenate((ex[0], ex[1:2*N-1:2]), axis=None)
  return s_dw, s_up

# v

In [32]:
def pot(v,N):
  p_up = np.zeros((N,2),dtype=np.float128)
  p_dw = np.zeros((N,2),dtype=np.float128)

  ran = np.arange(1,2*N,1,dtype=np.float128)
  ex=np.zeros(np.size(ran),dtype=np.float128)
  x=np.zeros(np.size(ran),dtype=np.float128)
  c = 2-v

  for ii in range(2*N-1):
    if ii == 0:
      x[ii]=c
      ex[ii] = 0
    else:
      x[ii] = x[ii-1]*c
      ex[ii] = ex[ii-1]
      if x[ii]>10:
        x[ii] = x[ii]/10
        ex[ii] = ex[ii]+1

  p_dw[:, 0] =  x[::2]
  p_dw[:, 1] =  ex[::2]

  ran = np.arange(1,N+1,1,dtype=np.float128)
  ex=np.zeros(np.size(ran),dtype=np.float128)
  x=np.zeros(np.size(ran),dtype=np.float128)
  c = 1-v

  for ii in range(N):
    if ii == 0:
      x[ii]= 1
      ex[ii] = 0
      continue
    if ii == 1:
      x[ii]=c*10
      ex[ii] = -1
    else:
      x[ii] = x[ii-1]*c
      ex[ii] = ex[ii-1]
      if x[ii]<1:
        x[ii] = x[ii]*10
        ex[ii] = ex[ii]-1

  p_up[:, 0] =  x[:N]
  p_up[:, 1] =  ex[:N]

  return p_dw, p_up

# Calculo ns


In [28]:
def calculate_ns(v,N, s_dw, s_up):
  p_dw, p_up = pot(v, N)
  ns_exp = (p_up[:,1]+s_up[:,1])-(p_dw[:,1]+2*s_dw[:,1])
  ns_val = (p_up[:,0]*s_up[:,0])/(p_dw[:,0]*np.power(s_dw[:,0], 2))
  ns = ns_val*np.power(10, ns_exp)*N
  ns[0] = N/(2-v)

  """ pruebas de valores
  for i in range(np.size(ns_exp)):
    #print(i+1, p_up[i,0], p_up[i,1], "-----", p_dw[i,0], p_dw[i,1]) #prueba para cada s de (1-v)^(s-1) y (2-v)^(2s-1) respectivamente
    #print(i+1, s_up[i,0], s_up[i,1], "-----", s_dw[i,0], s_dw[i,1]) #prueba para cada s de (2s-2)! y s! respectivamente
    #print(i+1, ns_val[i], ns_exp[i]) #prueba resultados en respresentacion ns_val * 10^(ns_exp) 
    print(i+1, ns[i]) #prueba resultados finales
  """
  return ns

# main 

In [45]:
def guerrillas(v, N):
  ns = np.zeros((np.size(v),N),dtype=np.float64)
  s_fac, s_2fac = fac(N)

  for i in range(np.size(v)):
    ns[i] = calculate_ns(v[i], N, s_fac, s_2fac)
  return ns 

In [50]:
v = np.array([0.01, 0.02, 0.03,0.04, 0.05])
N = 10000
ns = guerrillas(v,N)
print(ns)

[[5.02512563e+03 6.28124842e+02 2.09369660e+02 ... 2.20378594e-07
  2.20317933e-07 2.20257295e-07]
 [5.05050505e+03 6.31248718e+02 2.10394771e+02 ... 1.02799558e-07
  1.02763371e-07 1.02727199e-07]
 [5.07614213e+03 6.34370619e+02 2.11407835e+02 ... 2.81985766e-08
  2.81849889e-08 2.81714085e-08]
 [5.10204082e+03 6.37489481e+02 2.12407990e+02 ... 4.47467762e-09
  4.47169570e-09 4.46871588e-09]
 [5.12820513e+03 6.40604191e+02 2.13394339e+02 ... 4.03849647e-10
  4.03483231e-10 4.03117157e-10]]
